In [1]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
def line_iter(start, end):
    x0, y0 = start
    x1, y1 = end
    dx = abs(x1 - x0)
    dy = abs(y1 - y0)
    sx = 1 if x0 < x1 else -1
    sy = 1 if y0 < y1 else -1
    err = dx - dy
    while True:
        yield (x0, y0)
        if x0 == x1 and y0 == y1:
            break
        e2 = 2 * err
        if e2 > -dy:
            err -= dy
            x0 += sx
        if e2 < dx:
            err += dx
            y0 += sy


def find_last_point(img, start, end):
    last_black_point = None
    last_white_point = None
    answer=None
    for x, y in line_iter(start, end):
        if 0 <= x < img.shape[1] and 0 <= y < img.shape[0]:
            if img[y, x] == 0:
                last_black_point = (x, y)
            elif last_black_point and img[y, x] != 0:  # Check for a white point after the last black point
                answer=last_black_point
            
            if img[y,x] == 255:
                last_white_point = (x,y)
    return answer, last_white_point


def compute_areas(img):
    height, width = img.shape[:2]
    center= (int(width/2), int(height/2))
    subcut_area = 0
    for angle in np.arange(0, 360, granular_degree):
        theta = np.deg2rad(angle)
        end_x = int(center[0] + math.cos(theta) * width)
        end_y = int(center[1] + math.sin(theta) * height)
        bk,wh = find_last_point(img, center, (end_x, end_y))
        if bk and wh:
            subcut_area +=  0.5*((wh[0]-center[0])**2 +  (wh[1]-center[1])**2 - (bk[0]-center[0])**2 - (bk[1]-center[1])**2) * np.deg2rad(granular_degree)
    return subcut_area


def compute_white_area(img):
    total = 0
    for x in img:
        for y in x:
            if y==255:
                total+=1
    return total



In [4]:
!pwd

/Users/shashwatgupta/Library/CloudStorage/OneDrive-Personal/Stuff/Academics@IIT Kanpur/8th Semester Courses/UGP/Assingment/CV Algorithm Code/TB-Crohn


In [5]:
img_path = './Images/A.jpeg'
img = cv2.imread(img_path)
granular_degree = 0.05

kernel = np.ones((3,3),np.uint8)
thresh = cv2.dilate(cv2.erode(img,kernel,iterations = 1),kernel,iterations = 1)
img = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)

subcut_area = compute_areas(img)
total_area=compute_white_area(img)

ratio = (total_area / subcut_area) - 1
print("Ratio : ",ratio)

Ratio :  0.2951235072821905
